In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import os
from keras.applications.vgg16 import VGG16
import keras
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
import cv2
#import mathplotlib as plt
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"


1 Physical GPUs, 1 Logical GPUs


In [3]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [3]:
ph1=r"C:\Users\aanis\Desktop\plantvillage1\aa\segmented\tomato"
li=0
x=[]
y=[]
for i in os.listdir(ph1):
    lab=[0,0,0,0,0,0,0,0,0,0]
    lab[li]=1
    ph2=ph1+f'\{i}'
    #print(ph2)
    li+=1
    for j in os.listdir(ph2):
        #print(j)
        ph3=ph2+f'\{j}'
        #print(x,np.array(cv2.imread(ph3)).reshape(256*256*3))
        #print(lab)
        x.append(np.array(cv2.imread(ph3)))
        y.append(lab)
        

In [4]:
x=(np.array(x))
print(np.array(y).shape)

(18159, 10)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.2, random_state = 42) 
del x
del y
#print(x_train)

In [7]:
wpath=r"C:\Users\aanis\Downloads\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model=VGG16(weights=wpath,include_top=False,input_shape=(256,256,3))
base_model.trainable=False
classifier=keras.models.Sequential()
classifier.add(base_model)
classifier.add(Flatten())
#classifier.add(Dropout(0.2))
classifier.add(Dense(10,activation='softmax'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                327690    
Total params: 15,042,378
Trainable params: 327,690
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
BS=8
history = classifier.fit(aug.flow(x_train, y_train, batch_size=BS),
                         steps_per_epoch=len(x_train) // BS,
                         validation_data=(x_test, y_test),
                         epochs=15,
                         validation_steps=len(x_test) // BS,
                         )


###~94%

KeyboardInterrupt: 

In [10]:
train=history
acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
classifier.save("vgg16test2")

In [ ]:
reconstructed_model = keras.models.load_model("vgg16test1")
scores = reconstructed_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")